<a href="https://colab.research.google.com/github/GKurek94/Data-Science-projects/blob/main/Titanic_survival_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kaggle Titanic competition solution transfered to Google Collaboratory.

## Data Preprocessing 

Installing required library

In [ ]:
!pip install catboost


Importing the required libraries. Data load. First moment of preprocessing which is changing empty values to 0 or average.



In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split




train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

train_df['Cabin'] = train_df['Cabin'].fillna(0)
test_df['Cabin'] = test_df['Cabin'].fillna(0)
train_df['Age'] = train_df['Age'].fillna(train_df['Age'].mean())
test_df['Age'] = test_df['Age'].fillna(test_df['Age'].mean())
print(train_df.isnull().sum())
train_df.duplicated().sum()
#train_target = train_df.Survived
#train_data = train_df.drop('Survived', axis=1)
#test_df.head()



In [ ]:
from catboost import CatBoostClassifier

Splitting the data. 

In [ ]:
train_target = train_df.Survived
train_data = train_df.drop('Survived', axis=1)

For ease of project dropping column containing the name of passengers. 

In [ ]:
train_data = train_data.drop('Name', axis=1)
test_df = test_df.drop('Name', axis=1)

In [ ]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Pclass       891 non-null    int64  
 2   Sex          891 non-null    object 
 3   Age          891 non-null    float64
 4   SibSp        891 non-null    int64  
 5   Parch        891 non-null    int64  
 6   Ticket       891 non-null    object 
 7   Fare         891 non-null    float64
 8   Cabin        891 non-null    object 
 9   Embarked     889 non-null    object 
dtypes: float64(2), int64(4), object(4)
memory usage: 69.7+ KB


Another operation. There are around 150 cabins for survivors. For making calculations easier every cabin is equal to 1, which makes only 2 different values.

In [ ]:
train_data['Cabin'] = np.where(train_data['Cabin'] != 0, 1, 
                               train_data['Cabin'])
test_df['Cabin'] = np.where(test_df['Cabin'] != 0, 1, 
                               test_df['Cabin'])

0      0
1      1
2      0
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Cabin, Length: 891, dtype: object

In [ ]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Pclass       891 non-null    int64  
 2   Sex          891 non-null    object 
 3   Age          891 non-null    float64
 4   SibSp        891 non-null    int64  
 5   Parch        891 non-null    int64  
 6   Ticket       891 non-null    object 
 7   Fare         891 non-null    float64
 8   Cabin        891 non-null    object 
 9   Embarked     889 non-null    object 
dtypes: float64(2), int64(4), object(4)
memory usage: 69.7+ KB


In [ ]:
train_data['Ticket'].value_counts()

CA. 2343           7
1601               7
347082             7
3101295            6
347088             6
                  ..
244358             1
113767             1
SOTON/OQ 392089    1
347081             1
SW/PP 751          1
Name: Ticket, Length: 681, dtype: int64

Deleting ticket column, which shouldn't have impact into predicting survival.

In [ ]:
train_data = train_data.drop('Ticket', axis=1)
test_df = test_df.drop('Ticket', axis=1)

In [ ]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Pclass       891 non-null    int64  
 2   Sex          891 non-null    object 
 3   Age          891 non-null    float64
 4   SibSp        891 non-null    int64  
 5   Parch        891 non-null    int64  
 6   Fare         891 non-null    float64
 7   Cabin        891 non-null    object 
 8   Embarked     889 non-null    object 
dtypes: float64(2), int64(4), object(3)
memory usage: 62.8+ KB


Changing type of cabin column to numeric.

In [ ]:
train_data['Cabin'] = train_data['Cabin'].astype('int')
test_df['Cabin'] = test_df['Cabin'].astype('int')

In [ ]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Pclass       891 non-null    int64  
 2   Sex          891 non-null    object 
 3   Age          891 non-null    float64
 4   SibSp        891 non-null    int64  
 5   Parch        891 non-null    int64  
 6   Fare         891 non-null    float64
 7   Cabin        891 non-null    int64  
 8   Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(2)
memory usage: 62.8+ KB


Convert  variables into dummies.

In [ ]:
train_data = pd.get_dummies(train_data)
test_df = pd.get_dummies(test_df)

In [ ]:
test_df['Fare'] = test_df['Fare'].fillna(test_df['Fare'].mean())

Application which contains functions and all argorithms required to find best classifier for titanic survival prediction.

In [ ]:


def best_classifier(dicts):
        for e in dicts:
            if dicts[e] == max(dicts.values()):
                return e


# Logistic Regression
log_reg = LogisticRegression(random_state=0)
log_reg.fit(train_data, train_target)
y_pred_log = log_reg.predict(test_df)
log_score = log_reg.score(train_data, train_target)

# Multinomial Naive Bayes
classifier_multi = MultinomialNB()
classifier_multi.fit(train_data, train_target)
y_pred = classifier_multi.predict(test_df)
multi_score = classifier_multi.score(train_data, train_target)


# Decision Tree Classifier
classifier_tree = DecisionTreeClassifier(max_depth=2, random_state=0)
classifier_tree.fit(train_data, train_target)
y_pred_tree = classifier_tree.predict(test_df)
tree_score = classifier_tree.score(train_data, train_target)

# Linear SVC
classifier_svc = SVC(C=1.0, kernel='linear')
classifier_svc.fit(train_data, train_target)
y_pred_svc = classifier_svc.predict(test_df)
svc_score = classifier_svc.score(train_data, train_target)

# Catboost
cat_classifier = CatBoostClassifier(iterations=1000, learning_rate=0.01, depth=3)
cat_classifier.fit(train_data, train_target)
# Get predictions
cat_pred = cat_classifier.predict(test_df)
cat_score = cat_classifier.score(train_data, train_target)

# Dictionary of algorithm names and their values in that case
dict_of_algorithms = {
"LogisticRegression": log_score,
"MultinomialNB": multi_score,
"DecisionTreeClassifier": tree_score,
"LinearSVC": svc_score,
"CatBoost": cat_score}

results = {
"LogisticRegression": log_score,
"MultinomialNB": multi_score,
"DecisionTreeClassifier": tree_score,
"LinearSVC": svc_score,
"CatBoost": cat_score,
"Best Classifier": best_classifier(dict_of_algorithms),
"Best value": max(log_score, multi_score, tree_score, svc_score, cat_score)
}
print(results)

In [ ]:
submission = pd.DataFrame({
        "PassengerId": test_df["PassengerId"],
        "Survived": cat_pred
    })
submission

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [ ]:
submission.to_csv('submission.csv', index=False)